In [6]:
from jungeol import LoadUnivData
from jchan import CommercialArea

#숭실대학교, 서울대학교, 연세대학교, 중앙대학교, 동국대학교
ssu_data = LoadUnivData('숭실대학교') #LoadUnivData 객체 생성
ssu_gucode, ssu_dongname = ssu_data.loadUniv()

#병원, 백화점, 영화관 거리
ssu_cx, ssu_cy = ssu_data.loadXY()

soongsil_CA = CommercialArea(ssu_gucode, ssu_dongname, ssu_cx, ssu_cy) #CommercialArea 객체 생성

In [15]:
### df에서
# 처리 시간 엄청 오래 걸림
# bizesNm : 건물명
# indsLclsNm : 상권업종대분류명
# indsSclsCd : 상권업종소분류명
# lon : 경도
# lat : 위도
# 주로 위 데이터사용하면 될거야
### 객체.getDf().loc[0,:] 로 확인해봐도 좋고
df= soongsil_CA.getDf()
#학교별 상권 내 상가업소 데이터프레임
df = df.loc[df['indsLclsCd'] == 'Q'].head(10)
df

,bizesId,bizesNm,brchNm,indsLclsCd,indsLclsNm,indsMclsCd,indsMclsNm,indsSclsCd,indsSclsNm,ksicCd,...,bldMngNo,bldNm,rdnmAdr,oldZipcd,newZipcd,dongNo,flrNo,hoNo,lon,lat
3,11785500,맛있소면,,Q,음식,Q04,분식,Q04A01,라면김밥분식,I56194,...,1159010300103900000028846,,서울특별시 동작구 상도로 295,156836,06970,,1,,126.949531,37.501435
4,11789029,친구,,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,...,1159010200105010001020912,,서울특별시 동작구 사당로 8,156030,07027,,4,,126.955293,37.495314
5,11792428,보물쌈,상도동점,Q,음식,Q01,한식,Q01A08,족발/보쌈전문,I56111,...,1159010200101590170029313,계경목장,서울특별시 동작구 상도로 296-2,156031,07032,,2,,126.948935,37.501169
6,11793476,풍년집,,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1159010200105000016019770,,서울특별시 동작구 상도로 383,156030,07027,,1,,126.955106,37.495006
12,11911274,취향,,Q,음식,Q02,중식,Q02A00,중국음식/중국집,I56112,...,1159010200105010002019642,,서울특별시 동작구 사당로 6-1,156030,07027,,2,,126.955158,37.495331
13,11917318,지코바양념치킨,상도1호점,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,...,1159010300103670001029261,모가화장품,서울특별시 동작구 상도로37길 32,156836,06970,,1,,126.950228,37.500928
15,11931445,스타벅스숭실대입구역점,숭실대입구역점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1159010200104750004020804,재원어린이집,서울특별시 동작구 상도로 354,156030,07040,,1,,126.952705,37.496868
16,11953292,커피나무,,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1159010300103920000000001,김엔한스가,서울특별시 동작구 상도로 297-1,156836,06970,,1,,126.949681,37.501220
18,11975979,신전떡볶이,숭실대점,Q,음식,Q08,제과제빵떡케익,Q08A04,떡/한과집,I56191,...,1159010200104940001019381,,서울특별시 동작구 상도로 372-3,156030,07040,,,,126.953777,37.495418
19,11984505,유프,,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1159010200100070085028927,,서울특별시 동작구 흑석로 27,156030,06912,,2,,126.953720,37.505131


In [17]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import numpy as np


driver = webdriver.Chrome('chromedriver') 

df['naver_keyword'] = df['bizesNm']  # "%20"는 띄어쓰기를 의미합니다.
df['naver_map_url'] = ''


for i, keyword in enumerate(df['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        
        driver.get(naver_map_search_url)
        time.sleep(3.5)
        df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                df.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass


driver.quit()


# 이때 수집한 것은 완전한 URL이 아니라 URL에 들어갈 ID (data-cid 라는 코드명으로 저장된) 이므로, 온전한 URL로 만들어줍니다

df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']


# URL이 수집되지 않은 데이터는 제거합니다.
df = df.loc[~df['naver_map_url'].isnull()]

# 크롤링 에러가 떠서 'null'을 넣어 둔 데이터는 활용 의미가 없으므로 행 삭제를 해줘도 됩니다
# df = df.loc[~(df['naver_store_type'].str.contains('null'))]


# 별점 평균, 수 같은 데이터 역시 스트링 타입으로 크롤링이 되었으므로 numeric으로 바꿔줍니다.
# df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']] = df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']].apply(pd.to_numeric)

df

C:\Users\dali1\AppData\Local\Temp/ipykernel_25608/2353917122.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naver_keyword'] = df['bizesNm']  # "%20"는 띄어쓰기를 의미합니다.
C:\Users\dali1\AppData\Local\Temp/ipykernel_25608/2353917122.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naver_map_url'] = ''


이번에 찾을 키워드 : 0 / 8 행 친구


C:\Users\dali1\AppData\Local\Temp/ipykernel_25608/2353917122.py:26: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
C:\Users\dali1\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


이번에 찾을 키워드 : 1 / 8 행 보물쌈
이번에 찾을 키워드 : 2 / 8 행 풍년집
이번에 찾을 키워드 : 3 / 8 행 취향
이번에 찾을 키워드 : 4 / 8 행 지코바양념치킨
이번에 찾을 키워드 : 5 / 8 행 스타벅스숭실대입구역점
이번에 찾을 키워드 : 6 / 8 행 커피나무
이번에 찾을 키워드 : 7 / 8 행 신전떡볶이
이번에 찾을 키워드 : 8 / 8 행 유프


C:\Users\dali1\AppData\Local\Temp/ipykernel_25608/2353917122.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']


,bizesId,bizesNm,brchNm,indsLclsCd,indsLclsNm,indsMclsCd,indsMclsNm,indsSclsCd,indsSclsNm,ksicCd,...,rdnmAdr,oldZipcd,newZipcd,dongNo,flrNo,hoNo,lon,lat,naver_keyword,naver_map_url
4,11789029,친구,,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,...,서울특별시 동작구 사당로 8,156030,07027,,4,,126.955293,37.495314,친구,https://m.place.naver.com/restaurant/13015776
5,11792428,보물쌈,상도동점,Q,음식,Q01,한식,Q01A08,족발/보쌈전문,I56111,...,서울특별시 동작구 상도로 296-2,156031,07032,,2,,126.948935,37.501169,보물쌈,https://m.place.naver.com/restaurant/19763907
6,11793476,풍년집,,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,서울특별시 동작구 상도로 383,156030,07027,,1,,126.955106,37.495006,풍년집,https://m.place.naver.com/restaurant/31963119
12,11911274,취향,,Q,음식,Q02,중식,Q02A00,중국음식/중국집,I56112,...,서울특별시 동작구 사당로 6-1,156030,07027,,2,,126.955158,37.495331,취향,https://m.place.naver.com/restaurant/1816893717
13,11917318,지코바양념치킨,상도1호점,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,...,서울특별시 동작구 상도로37길 32,156836,06970,,1,,126.950228,37.500928,지코바양념치킨,https://m.place.naver.com/restaurant/1821108851
15,11931445,스타벅스숭실대입구역점,숭실대입구역점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,서울특별시 동작구 상도로 354,156030,07040,,1,,126.952705,37.496868,스타벅스숭실대입구역점,https://m.place.naver.com/restaurant/37391902
16,11953292,커피나무,,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,서울특별시 동작구 상도로 297-1,156836,06970,,1,,126.949681,37.501220,커피나무,https://m.place.naver.com/restaurant/37683228
18,11975979,신전떡볶이,숭실대점,Q,음식,Q08,제과제빵떡케익,Q08A04,떡/한과집,I56191,...,서울특별시 동작구 상도로 372-3,156030,07040,,,,126.953777,37.495418,신전떡볶이,https://m.place.naver.com/restaurant/1620683924
19,11984505,유프,,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,서울특별시 동작구 흑석로 27,156030,06912,,2,,126.953720,37.505131,유프,https://m.place.naver.com/restaurant/1496486342
